# Chapter 3. Building a Static Site Generator

- 말 그대로 static한 HTML 파일을 생성해 그 파일로만 서비스를 하는 것
    - 대표적으로 jekyll
        - 참고사이트: https://nolboo.github.io/blog/2013/10/15/free-blog-with-github-jekyll/
    - 서버 프로그래밍이 없기 때문에 github같은 곳에 올려서 많이 사용하고 있음
- [staticgen](https://www.staticgen.com/)
    - Python으로 만들어져 있는 Open source static site generator
        - [hyde](hyde.github.io)
        - [cactus](https://github.com/koenbok/Cactus/) using Python and Django template system.
- 중요 관점
    - Rapid prototyping

## 1. Layout(scaffolding) 잡기

## 2. Project Setting - prototypes.py

- p33
- **`'django.contrib.webdesign'`**
    - It provided the lorem template tag which is now included in the built-in tags. Simply remove 'django.contrib.webdesign' from INSTALLED_APPS and {% load webdesign %} from your templates.
    - https://docs.djangoproject.com/en/1.8/releases/1.8/#django-contrib-webdesign

## 3. urls 구조잡기

- p34

## 4. Template files - base.html, page.html(templates 폴더)

- `page.html` 파일에서 `base.html` 템플릿 파일을 상속받음
- 이 템플릿 파일이 생성될 파일의 가장 기본적인 골격이됨

## 5. Static Page Generator 준비 - pages 디렉토리 생성

## 6. settings 추가

- p37
- `SITE_PAGES_DIRECTORY`라는 이름으로 위에서 생성한 `pages` 디렉토리를 세팅

## 7. page를 렌더링시킬 view 작성

### 1) page(request, slug='index') 함수

- URL Dispatcher로부터 처음으로 Request되는 함수
- 페이지 이름과 페이지 내용(content)를 Context로 해서 템플릿 파일인 `page.html`로 렌더링해서 Response
- 페이지 내용(content)은 `get_page_or_404(file_name)`을 호출하여 불러옴

### 2) get_page_or_404(name) 함수

- 불러올 페이지가 있으면
    - 파일을 템플릿 형식으로 로딩해서 반환해줌
    - `with open(file_path, 'r') as f: page = Template(f.read())`
    - p37
- 불러올 페이지가 없으면
    -  `raise Http404('Page Not Found')` 에러 메시지

## 8. 템플릿 파일 page.html 작성하기

- rendering된 page content가 아래 부분처럼 include되는 형식
- `{% include page %}`

## 9. urls.py

```python
urlpatterns = (
    url(r'^(?P<slug>[\w./-]+)/$', page, name='page'),
    url(r'^$', page, name='homepage'),
)
```

- URL pattern이 무엇이든 상관없이 page 함수를 호출하게 되어있고 parameter는 slug

## 10. Styling

- p39 - p45
- Twitter Bootstrap을 이용
- `{% lorem %}` 템플릿 태그
    - `Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.`
- Navigation
    - 메뉴
        - Home
        - Contact
        - Login
    - 메뉴이름과 동일한 페이지를 만들 것이고, 해당 페이지가 템플릿 파일인 `page.html`의 block content로 넘어감
    - 이것을 나중에 build 시켜서 static한 페이지로 만드는 게 Static site generator
- **This is a prime example of how designers and developers can work in parallel.**
    - Contact와 Login의 서버 프로그래밍은 구현하지 않음

---

# Generating Static Content

## 11. Custome management command - build를 위한 setting

- `python prototypes.py 명령어`
- 예
    - `python manage.py runserver`
    - `python manage.py migrate`
- `python prototypes.py build`
- `_build`라는 디렉토리에 우리가 만든 index.html, contact.html, login.html에 대한 static content가 생성됨

1. settings.py의 INSTALLED_APPS에 등록된 app안에 `management/commands` 이름으로 디렉토리를 생성하고
2. 모듈 형태로 commnand를 만들 수 있음

## 12. build.py

**management/commands/build.py**

```python
import os
import shutil

from django.conf import settings
from django.core.management import call_command
from django.core.management.base import BaseCommand
from django.core.urlresolvers import reverse
from django.test.client import Client


def get_pages():
    for name in os.listdir(settings.SITE_PAGES_DIRECTORY):
        if name.endswith('.html'):
            yield name[:-5]


class Command(BaseCommand):
    help = "Build static site output."

    def handle(self, *args, **options):
        """Request pages and build output."""
        if os.path.exists(settings.SITE_OUTPUT_DIRECTORY):
            shutil.rmtree(settings.SITE_OUTPUT_DIRECTORY)
            
        os.mkdir(settings.SITE_OUTPUT_DIRECTORY)
        os.makedirs(settings.STATIC_ROOT)
        
        call_command('collectstatic', interactive=False, clear=True, verbosity=0)
        
        client = Client()
        
        for page in get_pages():
            url = reverse('page', kwargs={'slug': page})
            
            # URL을 요청해서 해당 content를 받아오기
            response = client.get(url)

            if page == 'index':
                output_dir = settings.SITE_OUTPUT_DIRECTORY
            else:
                # page와 동일한 이름으로 디렉토리 생성
                output_dir = os.path.join(settings.SITE_OUTPUT_DIRECTORY, page)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

            # 각 디렉토리에 index.html 생성
            with open(os.path.join(output_dir, 'index.html'), 'wb') as f:
                f.write(response.content)
```

- command의 경우 class Command(BaseCommand)를 해야하며 실질적인 logic은 def handle(...)에서 구현
- handle 함수에서 만들어져 있는 모든 페이지(index, contact, login)의 내용(content)을 가져와서(client.get(url) static page를 만들어주는 내용(f.write(response.content))

## 13. test

- `python -m http.server 9000`
- `python -m SingleHTTPServer 9000`
- npm 에서 live-server 이용 `live-server`

## 14. 위에서 처럼 전체 페이지가 아니라 하나의 페이지만 build시키기

```python
import os
import shutil

from django.conf import settings
from django.core.management import call_command
from django.core.management.base import BaseCommand, CommandError
from django.core.urlresolvers import reverse
from django.test.client import Client


def get_pages():
    for name in os.listdir(settings.SITE_PAGES_DIRECTORY):
        if name.endswith('.html'):
            yield name[:-5]


class Command(BaseCommand):
    help = "Build static site output."

    # TODO: 책에 없는 내용, 함수 추가
    def add_arguments(self, parser):
        """Entry point for subclassed commands to add custom arguments."""
        parser.add_argument('args', nargs='*')

    def handle(self, *args, **options):
        """Request pages and build output."""
        # TODO: argument 받을 수 있도록 if문 추가
        if args:
            pages = args
            available = list(get_pages())
            invalid = []
            
            # 사용자로부터 받은 arg를 validation
            for page in pages:
                if page not in available:
                    invalid.append(page)
            if invalid:
                msg = 'Invalid pages: {}'.format(', '.join(invalid))
                raise CommandError(msg)
        else:
            pages = get_pages()
            if os.path.exists(settings.SITE_OUTPUT_DIRECTORY):
                shutil.rmtree(settings.SITE_OUTPUT_DIRECTORY)
            os.mkdir(settings.SITE_OUTPUT_DIRECTORY)
            
        # FIXME: 책에 없는 내용, 들여쓰기와 exist_ok
        os.makedirs(settings.STATIC_ROOT, exist_ok=True)
        call_command('collectstatic', interactive=False, clear=True, verbosity=0)
        
        client = Client()
        
        # FIXME: get_pages() -> 위 if 문에서 설정한 pages
        for page in pages:
            url = reverse('page', kwargs={'slug': page})
            response = client.get(url)
            if page == 'index':
                output_dir = settings.SITE_OUTPUT_DIRECTORY
            else:
                output_dir = os.path.join(settings.SITE_OUTPUT_DIRECTORY, page)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            with open(os.path.join(output_dir, 'index.html'), 'wb') as f:
                f.write(response.content)
```

`python prototypes.py build index`

# Serving and Compressing Static Files

- Creating compressed static files is a another way for us to create fast page loads.
- static files를 hashing하는 방법도 있지만 소개만 하고 지워버림

## 15. django-compressor로 static파일 압축하기 위해 패키지 설치

`pip install django-compressor`

## 16. setting 추가

- INSTALLED_APPS
- STATICFILES_FINDERS

## 17. build.py 수정

```python
import os
import shutil

from django.conf import settings
from django.core.management import call_command
from django.core.management.base import BaseCommand, CommandError
from django.core.urlresolvers import reverse
from django.test.client import Client


def get_pages():
    for name in os.listdir(settings.SITE_PAGES_DIRECTORY):
        if name.endswith('.html'):
            yield name[:-5]


class Command(BaseCommand):
    help = "Build static site output."

    def add_arguments(self, parser):
        """Entry point for subclassed commands to add custom arguments."""
        parser.add_argument('args', nargs='*')

    def handle(self, *args, **options):
        """Request pages and build output."""
        
        # TODO: 추가 for compressing our static files
        settings.DEBUG = False
        settings.COMPRESS_ENABLED = True
        
        if args:
            pages = args
            available = list(get_pages())
            invalid = []
            for page in pages:
                if page not in available:
                    invalid.append(page)
            if invalid:
                msg = 'Invalid pages: {}'.format(', '.join(invalid))
                raise CommandError(msg)
        else:
            pages = get_pages()
            if os.path.exists(settings.SITE_OUTPUT_DIRECTORY):
                shutil.rmtree(settings.SITE_OUTPUT_DIRECTORY)
            os.mkdir(settings.SITE_OUTPUT_DIRECTORY)
        os.makedirs(settings.STATIC_ROOT, exist_ok=True)
        call_command('collectstatic', interactive=False, clear=True, verbosity=0)
        
        # TODO: 추가
        call_command('compress', interactive=False, force=True)
        
        client = Client()
        for page in pages():
            url = reverse('page', kwargs={'slug': page})
            response = client.get(url)
            if page == 'index':
                output_dir = settings.SITE_OUTPUT_DIRECTORY
            else:
                output_dir = os.path.join(settings.SITE_OUTPUT_DIRECTORY, page)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            with open(os.path.join(output_dir, 'index.html'), 'wb') as f:
                f.write(response.content)
```

## 18. base.html에서 css, js 파일들 compress tag 씌우기

```python
{% compress css %}
    <link rel="stylesheet" href="{% static 'css/bootstrap.min.css' %}">
    <link rel="stylesheet" href="{% static 'css/site.css' %}">
{% endcompress %}
```

- `python prototypes.py build`
- CACHE 폴더 생성됨
- 생성된 static 페이지를 보면 하나로 통합된 CACHE 파일을 불러오는 것을 확인할 수 있음
    - `<link rel="stylesheet" href="/static/CACHE/css/87cc9c5a5313.css" type="text/css" />`

# Generating Dynamic Content

## 19. pricing.html 추가

- 내용만 다를 뿐 똑같은 구조의 테이블이 한 페이지에 3개가 존재
- 즉 1x3

## 20. views.py 수정

- base.html에 meta tag설정
- pricing.html에 block context에 데이터만 들어감